This notebook can be run on Google Colab.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/ZKC19940412/water_ice_nep/colab-examples/example-1.ipynb)

In Colab, you can enable the GPU acceleration from `Edit` > `Notebook Settings` > `Hardware accelerator` > `GPU`.

# Install TDMDpy from Source ($\sim$ 4 min)

In [ ]:
%%capture
%cd /content/
# Fetch tdmdpy repo
! git clone https://github.com/ZKC19940412/tdmdpy.git

# Install tdmdpy from source
%cd tdmdpy
! pip install .

# Install pyNEP
! chmod +x get_pyNEP.sh
! ./get_pyNEP.sh
! echo "TDMDpy installation finishes!"

# Install [GPUMD](https://github.com/brucefan1983/GPUMD) from Source ($\sim$ 4 min)
- More instructions can be found : https://gpumd.org/installation.html

In [ ]:
%%capture
%cd /content/
! git clone https://github.com/brucefan1983/GPUMD.git
%cd /content/GPUMD/src/
! make -j 8
! echo "GPUMD installation finishes!"
%cd /content/

# Clean up Workspace and Fetch Nessary Files

In [ ]:
%cd /content/
! git clone https://github.com/ZKC19940412/water_ice_nep.git
! rm -r sample_data/

# Import Necessary Packages

In [ ]:
from ase.io import read, write
import numpy as np
from pylab import *
from pynvml import *
import matplotlib.pyplot as plt
from pynep.calculate import NEP
import random
from sklearn.metrics import mean_squared_error
from tdmdpy.mlps import single_point_energy_force_prediction

# Custom Function Define

In [ ]:
def set_fig_properties(ax_list):
    tl = 6
    tw = 2
    tlm = 4

    for ax in ax_list:
        ax.tick_params(which='major', length=tl, width=tw)
        ax.tick_params(which='minor', length=tlm, width=tw)
        ax.tick_params(which='both', axis='both',
                       direction='in', right=True, top=True,
                       left=True)

# Fitting NEP Potential

## Obtain Information about GPU Architecture for Particular colab Instance

In [ ]:
if __name__ == "__main__":

    nvmlInit()
    print("Driver Version:", nvmlSystemGetDriverVersion())
    deviceCount = nvmlDeviceGetCount()
    for i in range(deviceCount):
        handle = nvmlDeviceGetHandleByIndex(i)
        print("Device", i, ":", nvmlDeviceGetName(handle))

## Split Datasets into Training and Testing Sets

In [ ]:
if __name__ == "__main__":

    # Fix seed for replication
    random.seed(42)

    # Load in full data set
    configurations = read('water_ice_nep/nep-fitting/data.xyz', index=':')

    # Derive training and testing set indices as follow
    full_data_indices = list(np.arange(0, len(configurations), 1))
    training_set_indices = list(np.sort(random.sample(
        full_data_indices, 700)))
    testing_set_indices = list(set(full_data_indices).difference(
        set(training_set_indices)))


    # Write extended xyz for training and testing sets
    for training_set_indice in training_set_indices:
        write('train.xyz', configurations[training_set_indice], append=True)

    for testing_set_indice in testing_set_indices:
        write('test.xyz', configurations[testing_set_indice], append=True)

## Compose nep.in
- Format and instructions for parameters can be found: https://gpumd.org/tutorials/nep_tutorial.html.
- Batch size of 25 and generation of 1200 steps used here only serve as illustration purpose.

In [ ]:
%%writefile nep.in
version 3
type 2 H O
cutoff 8.0 6.0
n_max 8 8
basis_size 8 8
l_max 4 2 1
lambda_e 1.0
lambda_f 1.0
lambda_v 0.0
lambda_1 0.05
lambda_2 0.05
neuron 50
batch        25
generation   1200

## Peform Training ($\sim$ 15 min)

In [ ]:
! /content/GPUMD/src/nep

## Make Parity Plot to Visualize Training/Testing Pefromance

In [ ]:
if __name__ == "__main__":

    #  Set up Figure Styles
    aw = 2
    fs = 14
    font = {'size': fs}
    matplotlib.rc('font', **font)
    matplotlib.rc('axes', linewidth=aw)

    # Extract force data
    force_train = np.loadtxt('force_train.out')
    force_test = np.loadtxt('force_test.out')

    dft_force_train = force_train[:, :3]
    dft_force_test = force_test[:, :3]
    nep_force_train = force_train[:, 3:]
    nep_force_test = force_test[:, 3:]

    force_rmse_train = int(round(1000 * mean_squared_error(
        dft_force_train, nep_force_train) ** 0.5, 0))
    force_rmse_test = int(round(1000 * mean_squared_error(
        dft_force_test, nep_force_test ) ** 0.5, 0))

    figure(figsize=(10,8))
    subplot(2, 2, 1)
    set_fig_properties([gca()])
    scatter(dft_force_train, nep_force_train, s=20, facecolors='none',
            edgecolors='b', marker='o')
    plot([dft_force_train.min(), dft_force_train.max()],
         [dft_force_train.min(),
          dft_force_train.max()], '--', c='k', lw=2)
    gca().set_xticks(np.round(np.linspace(dft_force_train.min(),
                                          dft_force_train.max(), 3), 2))
    gca().set_yticks(np.round(np.linspace(dft_force_train.min(),
                                          dft_force_train.max(), 3), 2))
    text(0.2, 0.08, r'Training RMSE =' + str(
        force_rmse_train) + r' meV/$\AA$', c='k',
         fontsize=14, transform=gca().transAxes)
    xlabel(r'DFT force (eV/$\AA$)')
    ylabel(r'NEP force (eV/$\AA$)')
    subplot(2, 2, 3)
    set_fig_properties([gca()])
    scatter(dft_force_test, nep_force_test, s=20, facecolors='none',
            edgecolors='b', marker='s')
    plot([dft_force_test.min(), dft_force_test.max()],
         [dft_force_test.min(),
          dft_force_test.max()], '--', c='k', lw=2)
    gca().set_xticks(np.round(np.linspace(
        dft_force_test.min(), dft_force_test.max(), 3), 2))
    gca().set_yticks(np.round(np.linspace(
        dft_force_test.min(), dft_force_test.max(), 3), 2))
    text(0.2, 0.08, r'Testing RMSE =' + str(
        force_rmse_test) + r' meV/$\AA$', c='k',
         fontsize=14, transform=gca().transAxes)
    xlabel(r'DFT force (eV/$\AA$)')
    ylabel(r'NEP force (eV/$\AA$)')
    subplots_adjust(hspace=0.3)
    show()


# Single Point Energy & Force Predictions with Pretrain Model

In [ ]:
if __name__ == "__main__":

    # Uptake user define data path
    data_path = './test.xyz'

    # Derive number of configurations
    number_of_configurations = len(read(data_path, index = ':'))

    # Define NEP calculator
    nep_calculator = NEP('/content/water_ice_nep/nep-pre-train-model/nep.txt')

    print('ML Potential : Pretrained NEP')
    single_point_energy_force_prediction(number_of_configurations,
                                         nep_calculator,
                                         data_path, atoms_per_molecule=3,
                                         is_save_energy_and_force_file=True)